In [11]:
import librosa
import matplotlib.pyplot as plt
import librosa.display 
import numpy as np
import sox
import os
import soundfile as sf
from itertools import groupby
import pandas as pd

In [12]:
path_parent=r"F:\DLR\Keio\ALICE outputs\ALICE outputs\TC43-06_d1" #path to your parent folder (where you saved all data for each subject)
folders_parent=os.listdir(path_parent)
folders_parent.sort()

In [13]:
def outputall(path,paths,folder):
    if folder == '.DS_Store': #ignore the .DS_Store folder made by Mac OS
        return
    i=0 
    with open(path+'\\'+folder+'\\'+'diarization_output.rttm', 'r', encoding='utf-8') as f: 
        lines = f.readlines()
        line_n=len(lines)
        startp=np.zeros((line_n,1))
        duration=np.zeros((line_n,1))
        who=[np.nan]*line_n
    with open(path+'\\'+folder+'\\'+'diarization_output.rttm', 'r', encoding='utf-8') as f:
        for line in f:
            line=line.split(' ')
            startp[i]=line[3] #start of an utterance
            duration[i]=line[4] #duration of an utterance
            who[i]=line[7] #who is the speaker of an utterance
            i=i+1
    endtime=startp+duration # calculate the endtime of an utterance
    startend=np.c_[startp,endtime]
    for i in range(len(startend)): # Line labeled as 'SPEECH' is confused, remove them
        if who[i]=='SPEECH':
            startend[i,:]=np.nan
    startendclear=startend[~np.isnan(startend).all(axis=1), :]
    whoclear=[]
    for i in range(len(startend)):
        if who[i]!='SPEECH':
            whoclear.append(who[i])
            
    blocktime=np.c_[np.arange(6)*300+.001,np.arange(6)*300+300.001] # seperate each 30min data into 5min segments, and the estimations on 'high interactive moments' were conducted on each 5min segment
    
    count_t=np.zeros(len(blocktime))
    count_ttt=np.zeros(len(blocktime))
    for i in range(len(blocktime)): # we firstly choose the segments with KCHI vocalizations more than 20, named as 'highmoment'
        count=0
        for j in range(len(whoclear)):
            if whoclear[j]=='KCHI':
                if startendclear[j,0]>blocktime[i,0] and startendclear[j,1]<blocktime[i,1]:
                    count=count+1
        if count>20:
            count_ttt[i]=count
            count_t[i]=1

    highmoment=blocktime[np.where(count_t==1)[0]]
    
    count_ts=np.zeros(len(highmoment))
    count_tt=np.zeros(len(highmoment))
    
    for i in range(len(highmoment)): #Then we choose the segments from the 'highmoment' if the switches between KCHI and others (FEM/MAL/CHI) are more than 40
        counts=0
        for j in range(len(whoclear)-1):
            if startendclear[j,0]>highmoment[i,0] and startendclear[j,1]<highmoment[i,1]:
                if whoclear[j]=='KCHI':
                    if whoclear[j+1]!='KCHI': #if you want to count the switches between KCHI with particular interlocutor, please change this sentence as 'whoclear[j+1]!='FEM'' or 'whoclear[j+1]!='MAL'' or'whoclear[j+1]!='CHI''
                                              #if you want to count the switches between KCHI with multiple other interlocutors, e.g., FEM or MAL please change this sentence as 'whoclear[j+1]!='FEM' or whoclear[j+1]!='MAL''
                        counts+=1
                if whoclear[j]!='KCHI': #if you want to count the switches between KCHI with particular interlocutor, please change this sentence as 'whoclear[j+1]!='FEM'' or 'whoclear[j+1]!='MAL'' or'whoclear[j+1]!='CHI''
                                        #if you want to count the switches between KCHI with multiple other interlocutors, e.g., FEM or MAL please change this sentence as 'whoclear[j+1]!='FEM' or whoclear[j+1]!='MAL''
                    if whoclear[j+1]=='KCHI':
                        counts+=1
        if counts>40:
            count_tt[i]=counts
            count_ts[i]=1

    highswitch=highmoment[np.where(count_ts==1)[0]]
        
    output=highmoment
    outputs=highswitch
    allsp=pd.DataFrame(output)
    allsps=pd.DataFrame(outputs)
    allsp.columns=['starthm','endhm']
    allsps.columns=['starths','endhs']
    allsp.to_csv(paths+'\\'+folder+'\\'+'highmoment.csv') # There will be three outputs given, one is the highmoment (which 5min in the 30min)
    allsps.to_csv(paths+'\\'+folder+'\\'+'highswitch.csv') #The other is high switch (which 5min in the 30min)

    allss=np.c_[startendclear,whoclear]
    allssp=pd.DataFrame(allss)
    allssp.columns=['Starttime','Endtime','Speaker'] # The third is the speaker information
    allssp.to_csv(paths+'\\'+folder+'\\'+'VTC.csv')

In [14]:
path=path_parent #path to the ALICE folder
paths=r'F:\DLR\Keio\outputs' #save the outputs in the same folder or you could create new folders
for i in range(len(folders_parent)):
    folder2save=paths+'\\'+folders_parent[i]
    if not os.path.exists(folder2save): #check if folder exist. If not, create a the folder to save the output
        os.makedirs(folder2save)
    outputall(path,paths,folders_parent[i])